## Re-identification/De-identification 


In [ ]:
""" 
Read a configuration file containing column labels in csv. Strings are quasi-identifiers. 
"""
def read_config(config_file):
    with open(config_file) as file:
        identifiers = [(q) for q in file.read().split()]
    file.close()
    return sorted(identifiers)

### Import data

In [ ]:
import pandas as pd

Many entries contain NaN where the user did not enter information. Fill these values with 0 in order to filter them during analysis. 

In [ ]:
NA_FILL_VALUE = 0

In [ ]:
df_raw = pd.read_csv("mid_sample_set.csv", dtype='unicode')
# Set user id as index
df_raw.index = df_raw.user_id
df_raw = df_raw.drop('user_id', axis = 1)

In [ ]:
# Remove NA columns
original_columns = set(df_raw.columns.values)
df = df_raw.dropna(axis = 1, how = 'all').fillna(NA_FILL_VALUE)
new_columns = set(df.columns.values)
print("Removed columns", original_columns - new_columns)

In [ ]:
df.shape

Upon loading the dataset, there are 200,000 entries (users) and 87 fields (identifiers) for each row entry.

### Direct Identifiers
- Can uniquely identify an individual and should be removed
- This includes ip


In [ ]:
df = df.drop('ip', axis = 1)

### Quasi-Identifiers
- Can uniquely identify an individual when linked to other datasets. 
- These include: 'user_id', 'countryLabel', 'continent', 'city', 'region', 'subdivision', 'postalCode', 'LoE', 'YoB', 'gender', 'nforum_posts', 'nforum_votes', 'nforum_endorsed', 'nforum_threads', 'nforum_comments', 'nforum_pinned', and 'nforum_events', and are listed as their corresponding column index in the configuration file.
- Redundant quasi-identifiers are not included 
- course_id was dropped 
- Create a version of the dataset that only contains the quasi-identifiers.

In [ ]:
quasi_identifiers = read_config("config_file.txt")
df_quasi = df.loc[:,quasi_identifiers]
labels = list(df_quasi.columns.values)
# Remove user_id because it's not a quasi-identifier, just a key
quasi_identifier_labels = list(set(labels) - set(["user_id"]))

In [ ]:
""" 
Suppresses groups that Calculates the percentage of students who completed the course. Inputs are the dataframe,
quasi-identifiers and level of k-anonmity 
"""
def getAnonCompletionRates(df, quasi_identifier_labels, k):
    if 'course_id' not in quasi_identifier_labels:
        quasi_identifier_labels.append('course_id')
    # Creates groupings of quasi identifiers and finds size of each set
    k_anonymous = df.groupby(quasi_identifier_labels)\
                  .size().reset_index(name = 'ct').set_index(quasi_identifier_labels)
    k_anonymous = k_anonymous[k_anonymous.ct >= k]
    
    # Add completion status for each student 
    completedCount = df.groupby(quasi_identifier_labels + ['completed'])\
                       .size()\
                       .reset_index(name = "completedCount")\
                       .set_index(quasi_identifier_labels)
                
    # Find size of altered dataset after k-anonymization 
    k_anonymous = k_anonymous.reset_index()\
                  .merge(completedCount[completedCount.completed == 'True'].reset_index(), how = "left")
        
    print("Size of %d-anonymous dataset: %d" % (k, k_anonymous.ct.sum()))
    
    # Aggregate statuses and find overall completion rate 
    k_AnonTotalCounts = k_anonymous.groupby('course_id')["ct"].sum()\
                                   .reset_index(name = "numStudents")\
                                   .set_index('course_id')
    
    completionRates = ((k_anonymous.set_index('course_id').completedCount.fillna(0)
                        / k_AnonTotalCounts.numStudents).fillna(0) * 100)\
                        .reset_index(name = "completionRate")\
                        .sort_values(by = "completionRate", ascending = False)
    
    if len(k_anonymous) == 0: 
        completionRates.completionRate = 0
    
    return completionRates 

When k = 3

In [ ]:
completionRate3Anon = getAnonCompletionRates(df, quasi_identifier_labels, 3)

In [ ]:
completionRate3Anon.head()

When k = 4

In [ ]:
completionRate4Anon = getAnonCompletionRates(df, quasi_identifier_labels, 4)

In [ ]:
completionRate4Anon.head()

When k = 5

In [ ]:
completionRate5Anon = getAnonCompletionRates(df, quasi_identifier_labels, 5)

In [ ]:
completionRate5Anon.head()

### Synthetic Records
- Make the data k-anonymous
- Find the number of synthetic records needed for each case
- Compute completion rates and compare to dataset without synthetic records

When k = 3

In [ ]:
"""
Adds artifical rows of data in order to acheive desired k-anonmity level. 
Each synthetic row is a duplicate of row data that is not yet k-anonymous.
"""
def addSyntheticRows(df, quasi_identifier_labels, k):
    synthetic_k_anon_df = df.reset_index()
    # Create groupings based on quasi identifiers and count number of students in each group
    not_k_anonymous = synthetic_k_anon_df.groupby(quasi_identifier_labels)\
                      .size().reset_index(name = 'studentCount')
        
    # Assign groupings where studentCount is less than desired level for k-anonymity 
    not_k_anonymous = not_k_anonymous[not_k_anonymous.studentCount < k]

    # Duplicate rows where groupings of quasi-identifiers are not k-anonymous
    for i in range(k):
        rowsToAdd = not_k_anonymous[not_k_anonymous.studentCount == i]
        if len(rowsToAdd) > 0:
            for j in range(k - i):
                synthetic_k_anon_df = synthetic_k_anon_df.append(rowsToAdd, ignore_index = True)
            
    # Calculate difference between dataset and k-anonymized dataset with synthetic rows
    rowsAdded = synthetic_k_anon_df.shape[0] - len(df)
    print("Size of synthetic dataset: %d\nSynthetic Rows Added: %d" % (len(synthetic_k_anon_df), rowsAdded))
    # Drop the studentCount column from the dataset we return because df_quasi does not
    # have it.  This column is an artifact of when we grouped synthetic_k_anon_df above
    synthetic_k_anon_df = synthetic_k_anon_df.drop("studentCount", axis = 1)
    return synthetic_k_anon_df
    

Add synthetic data when k = 3

In [ ]:
synthetic3AnonDf = addSyntheticRows(df, quasi_identifier_labels, 3)

Add synthetic data when k = 4

In [ ]:
synthetic4AnonDf = addSyntheticRows(df, quasi_identifier_labels, 4)

Add synthetic data when k = 5

In [ ]:
synthetic5AnonDf = addSyntheticRows(df, quasi_identifier_labels, 5)

### K-Anonymity 
- Generalization
    - YoB, nform_* 
- Blurring
    - Last 3 digits of postal_code
- Suprression 
    - Remaining 


In [ ]:
 df_quasi_copy = df.copy()

In [ ]:
def blurring(df, col, lenCol):
    df_quasi_blur = df
    # Blurr the last 3 elements of string with stars
    df_quasi_blur.loc[:,col] = df_quasi_blur.loc[:,col].astype(str).str[0:int(lenCol-3)] + "*" * 3;
    return df_quasi_blur

In [ ]:
blurring(df_quasi_copy, "postalCode", 5);

In [ ]:
def generalization(df, colChange, bucketSize, maxVal):
    col = colChange
    df_quasi_gen = df
    # Calculate number of categories with maximum value in column and size of each category interval 
    binNum = int(maxVal / bucketSize)
    # Create list of categories 
    bins = [bucketSize*i for i in range(NA_FILL_VALUE-1, binNum)]
    # Convert data to integers and assign each value to specified list of categories 
    df_quasi_gen.loc[:,col] = df_quasi_gen[col].astype(int)
    df_quasi_gen.loc[:,col] = pd.cut(df_quasi_gen[col], bins)
    return df_quasi_gen

In [ ]:
r = generalization(df_quasi_copy, "YoB", 10, 2019)
r = generalization(r, "nforum_posts", 10, 470)
r = generalization(r, "nforum_votes", 10, 640)
r = generalization(r, "nforum_endorsed", 10, 50)
r = generalization(r, "nforum_threads", 10, 150)
r = generalization(r, "nforum_comments", 10, 450)
r = generalization(r, "nforum_pinned", 10, 20)

### Q5. Compare the number of students who complete and explore the course in the original and in the k-anonymous sets. What does this tell you about the process of de-identification?
The process of de-identification adds noise to the dataset, eliminates statistically relevant information, increases the size of the dataset and decreases utility. The completion rates in the k-anonymous sets were <span style="color:red">[get the numbers]</span>.

### L-Diversity
- Used to determine distinguishability among indistunghisable quasi-identifiers but distinguishable sensitive attributes
- Sensitive attributes include: grade
- Determine level of l-diversity in order to strengthen k-anonymity

In [ ]:
lDiverse = r
# Add column of sensitive values to dataset that is k-anonymous
lDiverse["grade"] = df["grade"]

In [ ]:
lDiverseLevels = lDiverse.groupby(quasi_identifier_labels)["grade"].nunique().reset_index(name = "l-diversity-grade")
# Find level of diversity (number of unique grades) for each grouping of quasi-identifiers
lDiverseLevels["l-diversity-grade"].head()

In [ ]:
lDiverseLevels["l-diversity-grade"].unique()

In [ ]:
# Add completed column in order to find completion rate on modified dataset 
df_quasi["completed"] = df["completed"]

In [ ]:
completionRate4Anon_GenBlurSuppress = getAnonCompletionRates(df_quasi, quasi_identifier_labels, 4)

In [ ]:
completionRate5Anon_GenBlurSuppress = getAnonCompletionRates(df_quasi, quasi_identifier_labels, 5) 

### Q6. What fields in the data set might be considered "sensitive" 
A student's grade may be considered sensitive
### In your datasets in 5), what level of l-diversity do the data sets have with respect to these fields? How might you get to a higher level of l-diversity?
The level od l-diversity for each set of quasi-identifiers is indicated under the "l-diversity-grade" column in lDiverseLevels. Further processing through generalization and blurring would achieve a higher level of l-diversity.

